In [ ]:
!pip install -q h5py pyyaml

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
# 학습데이터, 테스트 데이터 로드
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [ ]:
#모델 생성
def create_model():
    model = tf.keras.models.Sequential([
                                        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                                        keras.layers.Dropout(0.2),
                                        keras.layers.Dense(10, activation='softmax')])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

    return model

model = create_model()
model.summary()

In [ ]:
# 체크포인트 콜백 설정(다시 훈련 X, 모델을 재사용, 훈련 과정이 중지된 경우 이어서 사용)
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
model = create_model()
model.fit(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels), callbacks = [cp_callback])


In [ ]:
!ls {checkpoint_dir}

In [ ]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

In [ ]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

In [ ]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,period=5) # 다섯 번째 에포크마다 가중치를 저장
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,epochs = 50, callbacks = [cp_callback],validation_data = (test_images,test_labels),verbose=0)    
    

In [ ]:
! ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

In [ ]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint')

# 가중치 복원
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

In [ ]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

In [ ]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

In [ ]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

In [ ]:
# saved_model 사용
# keras.experimental에서 export_saved_model이 정상 작동하지 않고 있어 아래 부분은 정상 진행 하지 못하였습니다.
import time
from tensorflow import keras
saved_model_path = "./saved_models/{}".format(int(time.time()))
tf.keras.experimental.export_saved_model(model, saved_model_path)